In [6]:
import cv2
import numpy as np
import tkinter as tk
from tkinter import filedialog
from scipy.ndimage import interpolation as inter


In [196]:
class OCRPreprocessing:
    def __init__(self, input_file = "",  output_file = "", input_res =(1200,1000)):
        self.input_file = input_file
        self.input_res = input_res
        self.output_file = output_file
        self.title = "Open OCR Image"
        self.filetypes = (
        ('Image files', '*.jpg'),
        ('Image files', '*.png'),
        ('All files', '*.*'))
        
    def openImageFileDialog(self):

        root = tk.Tk()
        root.withdraw()
        
        fileSelect = filedialog.askopenfilename(title = self.title, initialdir='/',
        filetypes = self.filetypes)
        self.input_file = fileSelect
        return 
    
    def openInputImage(self):
        print("open image",self.input_file)
        self.input_mat = cv2.imread(self.input_file)
        
        
    def findImageScore(self, arr, angle):
        data = inter.rotate(arr, angle, reshape=False, order=0)
        hist = np.sum(data, axis=1)
        score = np.sum((hist[1:] - hist[:-1]) ** 2)
        return hist, score
    
    def findImageAngle(self,input_image):
        delta = 1
        limit = 5
        angles = np.arange(-limit, limit+delta, delta)
        scores = []
        for angle in angles:
            hist, score = self.findImageScore(input_image, angle)
            scores.append(score)
        best_score = max(scores)
        best_angle = angles[scores.index(best_score)]
        return best_angle
    
    def imageSmoothening(self,img, binaryThres = 180):
        ret1, th1 = cv2.threshold(img, binaryThres, 255, cv2.THRESH_BINARY)
        ret2, th2 = cv2.threshold(th1, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
        blur = cv2.GaussianBlur(th2, (1, 1), 0)
        ret3, th3 = cv2.threshold(blur, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
        return th3
    
    def removeNoiseAndSmooth(self, img, textSizeSmoothness):
        filtered = cv2.adaptiveThreshold(img,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY,11,2)
        kernel = np.ones((1, 1), np.uint8 )
        opening = cv2.morphologyEx(filtered, cv2.MORPH_OPEN, kernel)
        closing = cv2.morphologyEx(opening, cv2.MORPH_CLOSE, kernel)
        img = self.imageSmoothening(img, textSizeSmoothness)
        or_image = cv2.bitwise_or(img, filtered)
#         cv2.imshow("remove noise image", or_image)
        return or_image
       
    
    def preprocessingImage(self, denoiseFilter = 10, textSizeSmoothness = 280, rotateImage = False):
        self.openInputImage()
        #resize image to have a standard size for preprocessing
        resizedImg = cv2.resize(self.input_mat, self.input_res, interpolation = cv2.INTER_AREA)
        #denoise image
        filterImg = cv2.bilateralFilter(resizedImg,9,75,150)
        if denoiseFilter > 0:
            filterImg = cv2.fastNlMeansDenoisingColored(filterImg, None, denoiseFilter, denoiseFilter, 7, 15) 
        #convert into grayscale image
        grayImg = cv2.cvtColor(filterImg, cv2.COLOR_BGR2GRAY)
        #remove noise and smoothen the imgae
        output = self.removeNoiseAndSmooth(grayImg, textSizeSmoothness)
        if rotateImage :
            outputImg = inter.rotate(output, self.findImageAngle(output) , reshape=False, order=0)
#         cv2.imshow("denoise", filterImg)
#         cv2.imshow("gray", grayImg)
        cv2.imwrite(self.output_file, output)
        return output
        
    
        
        
    

In [197]:
ocr= OCRPreprocessing("", "out.jpg")


In [198]:
ocr.openImageFileDialog()


In [199]:

cv2.imshow("output",ocr.preprocessingImage(2, 270))
cv2.waitKey(-1)

open image C:/Users/goldf/Desktop/projects/sample.JPG


-1